## **Importing Libraries**

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.metrics import error_rate

# **Stage 1**

## **Data Setup and Preview**

In [3]:
data_path = Path.home()/'notebooks/programming/data/watch_bot/'
csv_path_1 = Path.home()/'notebooks/programming/projects/watch_bot/data_prep/file_attribs_watch_bot.csv'
csv_path_2 = Path.home()/'notebooks/programming/projects/watch_bot/data_prep/chrono24_dial_visibility_most_uncertain.csv'


In [4]:
df1 = pd.read_csv(csv_path_1)
df1.head()

,image_size_x,image_size_y,name,dial_visibility,like,image_quality
0,960,960,nomos_1/nomos/_without_model/nomos_@_the_loupe...,1.0,1.0,1.0
1,960,960,nomos_1/nomos/_without_model/nomos_@_the_loupe...,1.0,0.0,1.0
2,960,960,nomos_1/nomos/_without_model/nomos_@_the_loupe...,0.0,0.0,0.0
3,720,960,nomos_1/nomos/_without_model/nomos_beta_11156_...,1.0,0.0,1.0
4,720,960,nomos_1/nomos/_without_model/nomos_beta_11156_...,1.0,0.0,1.0


In [5]:
df1.shape

(11973, 6)

In [6]:
df2 = pd.read_csv(csv_path_2)
df2.head()

,image_size_x,image_size_y,name,dial_visibility,dial_visibility_p_0,dial_visibility_p_1,like,like_p_0,like_p_1,image_quality
0,640,480,chrono24_dial_visibility_most_uncertain/a._lan...,1.0,0.473409,0.526591,-1.0,-1,-1,-1.0
1,1200,718,chrono24_dial_visibility_most_uncertain/a._lan...,0.0,0.480346,0.519654,-1.0,-1,-1,-1.0
2,640,480,chrono24_dial_visibility_most_uncertain/a._lan...,0.0,0.471942,0.528058,-1.0,-1,-1,-1.0
3,640,480,chrono24_dial_visibility_most_uncertain/a._lan...,0.0,0.477966,0.522034,-1.0,-1,-1,-1.0
4,640,480,chrono24_dial_visibility_most_uncertain/a._lan...,0.0,0.471942,0.528058,-1.0,-1,-1,-1.0


In [7]:
df2.shape

(17379, 10)

In [8]:
df = pd.concat([df1, df2], ignore_index=True)
df = df.dropna(axis=1, how='any')
df

,image_size_x,image_size_y,name,dial_visibility,like,image_quality
0,960,960,nomos_1/nomos/_without_model/nomos_@_the_loupe...,1.0,1.0,1.0
1,960,960,nomos_1/nomos/_without_model/nomos_@_the_loupe...,1.0,0.0,1.0
2,960,960,nomos_1/nomos/_without_model/nomos_@_the_loupe...,0.0,0.0,0.0
3,720,960,nomos_1/nomos/_without_model/nomos_beta_11156_...,1.0,0.0,1.0
4,720,960,nomos_1/nomos/_without_model/nomos_beta_11156_...,1.0,0.0,1.0
...,...,...,...,...,...,...
29347,960,960,chrono24_dial_visibility_most_uncertain/zenith...,1.0,-1.0,-1.0
29348,960,960,chrono24_dial_visibility_most_uncertain/zenith...,1.0,-1.0,-1.0
29349,767,960,chrono24_dial_visibility_most_uncertain/zenith...,0.0,-1.0,-1.0
29350,720,960,chrono24_dial_visibility_most_uncertain/zenith...,0.0,-1.0,-1.0


In [ ]:
bs = 64
size = 128
xform = 